In [ ]:
import spacy
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

In [ ]:
nlp=spacy.load("en_core_web_sm")

In [ ]:
data=pd.read_csv("/content/Reviews.csv", on_bad_lines="skip",engine="python")
data.head()

,Id,ProductId,UserId,ProfileName,HelpfulnessNumerator,HelpfulnessDenominator,Score,Time,Summary,Text
0,1,B001E4KFG0,A3SGXH7AUHU8GW,delmartian,1,1,5,1303862400,Good Quality Dog Food,I have bought several of the Vitality canned d...
1,2,B00813GRG4,A1D87F6ZCVE5NK,dll pa,0,0,1,1346976000,Not as Advertised,Product arrived labeled as Jumbo Salted Peanut...
2,3,B000LQOCH0,ABXLMWJIXXAIN,"Natalia Corres ""Natalia Corres""",1,1,4,1219017600,"""Delight"" says it all",This is a confection that has been around a fe...
3,4,B000UA0QIQ,A395BORC6FGVXV,Karl,3,3,2,1307923200,Cough Medicine,If you are looking for the secret ingredient i...
4,5,B006K2ZZ7K,A1UQRSCLF8GW1T,"Michael D. Bigham ""M. Wassir""",0,0,5,1350777600,Great taffy,Great taffy at a great price. There was a wid...


In [ ]:
data['Text']

,Text
0,I have bought several of the Vitality canned d...
1,Product arrived labeled as Jumbo Salted Peanut...
2,This is a confection that has been around a fe...
3,If you are looking for the secret ingredient i...
4,Great taffy at a great price. There was a wid...
...,...
568449,Great for sesame chicken..this is a good if no...
568450,I'm disappointed with the flavor. The chocolat...
568451,"These stars are small, so you can give 10-15 o..."
568452,These are the BEST treats for training and rew...


In [ ]:
data.isna().sum()

,0
Id,0
ProductId,0
UserId,0
ProfileName,26
HelpfulnessNumerator,0
HelpfulnessDenominator,0
Score,0
Time,0
Summary,27
Text,0


In [ ]:
data.dropna(inplace=True)

In [ ]:
data.isna().sum()

,0
Id,0
ProductId,0
UserId,0
ProfileName,0
HelpfulnessNumerator,0
HelpfulnessDenominator,0
Score,0
Time,0
Summary,0
Text,0


In [ ]:
processed_data=data[:1000]

In [ ]:
def cleaned_text(text):
  doc = nlp(text.lower())
  lemmas=[]
  for tokens in doc:
    if tokens.is_alpha and not tokens.is_stop:
      lemmas.append(tokens.lemma_)
  return " ".join(lemmas)

In [ ]:
import re
data['cleaned_text']=data['Text'].apply(cleaned_text)

KeyboardInterrupt: 

In [ ]:
import nltk
nltk.download('stopwords')


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [ ]:
from nltk.corpus import stopwords
stop_words = set(stopwords.words('english'))


In [ ]:
import re
from nltk.corpus import stopwords

stop_words = set(stopwords.words('english'))

def cleaned_text(text):
    text = text.lower()
    text = re.sub(r'[^a-z\s]', '', text)
    tokens = text.split()
    tokens = [word for word in tokens if word not in stop_words]
    return " ".join(tokens)

data['cleaned_text'] = data['Text'].astype(str).apply(cleaned_text)
print(data['cleaned_text'].head())


0    bought several vitality canned dog food produc...
1    product arrived labeled jumbo salted peanutsth...
2    confection around centuries light pillowy citr...
3    looking secret ingredient robitussin believe f...
4    great taffy great price wide assortment yummy ...
Name: cleaned_text, dtype: object


In [ ]:
tfidf=TfidfVectorizer(max_features=5000)
tfidf_fit=tfidf.fit_transform(texts)

In [ ]:
print(tfidf_fit.shape)

(568401, 5000)


In [ ]:
def preprocess_query(text):
    text = text.lower()
    text = re.sub(r'[^a-z\s]', '', text)
    tokens = text.split()
    tokens = [word for word in tokens if word not in stop_words]
    return " ".join(tokens)

In [ ]:
query = "great product with fast shipping"
cleaned_query = preprocess_query(query)
print("Cleaned query:", cleaned_query)

Cleaned query: great product fast shipping


In [ ]:
query_vec = tfidf.transform([cleaned_query])
print(query_vec.shape)

(1, 5000)


In [ ]:
def retrive_reviews(query,k=5):
  cleaned_query=query.lower()
  cleaned_query=cleaned_text(query)
  query_vec=tfidf.transform([cleaned_query])
  similarity_score=cosine_similarity(query_vec,tfidf_fit).flatten()
  top_indices=similarity_score.argsort()[::-1][:k]

  results=[]
  for i in top_indices:
    results.append({
        "original":data.iloc[i]['Text'],
        "cleaned":data.iloc[i]['cleaned_text'],
        "score":float(similarity_score[i])
    })
  return results

In [ ]:
for res in retrive_reviews("great product with fast fishing", k=3):
    print("Original:", res["original"])
    print("Cleaned :", res["cleaned"])
    print("Score   :", res["score"])
    print("-" * 80)


Original: Great product ... great price .. shipped fast!Great product ... great price .. shipped fast!Great product ... great price .. shipped fast!Great product ... great price .. shipped fast!Great product ... great price .. shipped fast!Great product ... great price .. shipped fast!Great product ... great price .. shipped fast!Great product ... great price .. shipped fast!Great product ... great price .. shipped fast!Great product ... great price .. shipped fast!Great product ... great price .. shipped fast!
Cleaned : great product great price shipped fastgreat product great price shipped fastgreat product great price shipped fastgreat product great price shipped fastgreat product great price shipped fastgreat product great price shipped fastgreat product great price shipped fastgreat product great price shipped fastgreat product great price shipped fastgreat product great price shipped fastgreat product great price shipped fast
Score   : 0.7228025328809055
-------------------------